In [2]:
# plant_disease_pipeline.py
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
import warnings
import traceback

# Optional: if you use kagglehub; keep it but handle missing package gracefully
try:
    import kagglehub
except Exception:
    kagglehub = None

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

warnings.filterwarnings("ignore")

# --- GPU and TensorFlow Optimizations ---
def setup_gpu():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU available: {len(gpus)} GPU(s) configured.")
            # Mixed precision
            try:
                tf.keras.mixed_precision.set_global_policy('mixed_float16')
                print("✅ Mixed precision policy set to 'mixed_float16'.")
            except Exception as e:
                print(f"⚠ Could not set mixed precision policy: {e}")
            # XLA JIT
            try:
                tf.config.optimizer.set_jit(True)
                print("✅ XLA JIT enabled.")
            except Exception as e:
                print(f"⚠ Could not enable XLA: {e}")
        except RuntimeError as e:
            print(f"⚠ GPU setup error: {e}")
    else:
        print("ℹ Using CPU. For faster training, use a GPU environment (e.g., Colab, Kaggle).")

setup_gpu()

# Directory to save models and results
OUTPUT_DIR = "plant_disease_results_DL"
os.makedirs(OUTPUT_DIR, exist_ok=True)

class PlantDiseaseDL:
    def __init__(self, base_path, img_size=(224, 224)):
        """
        Fixed constructor name and basic initializations.
        """
        self.base_path = base_path
        self.img_size = tuple(img_size)
        self.class_names = []
        self.label_map = {}

    def load_images(self):
        images, labels = [], []
        if not os.path.exists(self.base_path):
            raise FileNotFoundError(f"Dataset path '{self.base_path}' not found!")
        # find class subdirectories (ignore hidden files)
        self.class_names = sorted([
            d for d in os.listdir(self.base_path)
            if os.path.isdir(os.path.join(self.base_path, d)) and not d.startswith('.')
        ])
        self.label_map = {name: idx for idx, name in enumerate(self.class_names)}
        print(f"Found {len(self.class_names)} classes: {self.class_names}")
        for class_name in self.class_names:
            class_path = os.path.join(self.base_path, class_name)
            for filename in os.listdir(class_path):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(class_path, filename)
                    try:
                        img = cv2.imread(img_path)
                        if img is None:
                            # corrupted or unreadable file
                            print(f"⚠ Warning: could not read image {img_path}")
                            continue
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img = cv2.resize(img, self.img_size)
                        images.append(img)
                        labels.append(self.label_map[class_name])
                    except Exception as e:
                        print(f"Error loading {img_path}: {e}")
        images = np.array(images, dtype=np.uint8)
        labels = np.array(labels, dtype=np.int32)
        print(f"Loaded {len(images)} images.")
        return images, labels

    def plot_learning_curve(self, history, model_name):
        plt.style.use('seaborn-v0_8-whitegrid')
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))
        # handle mixed-precision producing 'loss' vs 'val_loss' etc.
        for k in ['accuracy', 'val_accuracy']:
            if k not in history.history:
                history.history[k] = []
        ax1.plot(history.history.get('accuracy', []), label='Train Accuracy', marker='o')
        ax1.plot(history.history.get('val_accuracy', []), label='Val Accuracy', marker='x')
        ax1.set_title(f'{model_name} Accuracy')
        ax1.legend(loc='best')

        ax2.plot(history.history.get('loss', []), label='Train Loss', marker='o')
        ax2.plot(history.history.get('val_loss', []), label='Val Loss', marker='x')
        ax2.set_title(f'{model_name} Loss')
        ax2.legend(loc='best')

        save_path = os.path.join(OUTPUT_DIR, f'{model_name}_learning_curve.png')
        plt.tight_layout()
        plt.savefig(save_path, dpi=300)
        plt.close()
        print(f"✅ Learning curve saved to {save_path}")

    def generate_evaluation_report(self, y_true, y_pred, model_name):
        report = classification_report(y_true, y_pred, target_names=self.class_names, zero_division=0)
        report_path = os.path.join(OUTPUT_DIR, f'{model_name}_classification_report.txt')
        with open(report_path, 'w') as f:
            f.write(report)
        print(f"✅ Classification report saved to {report_path}")

        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(max(8, len(self.class_names)), max(6, len(self.class_names))))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=self.class_names, yticklabels=self.class_names, annot_kws={"size": 8})
        plt.title(f'Confusion Matrix: {model_name}')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.tight_layout()
        cm_path = os.path.join(OUTPUT_DIR, f'{model_name}_confusion_matrix.png')
        plt.savefig(cm_path, dpi=300)
        plt.close()
        print(f"✅ Confusion matrix saved to {cm_path}")

    def build_efficientnet_b4_model(self, num_classes):
        # Load EfficientNetB4 base. If weights fail to download, provide helpful message.
        try:
            base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=self.img_size + (3,))
        except Exception as e:
            raise RuntimeError(f"Failed to load EfficientNetB4 weights: {e}\n"
                               "Check your internet connection or try a smaller model (e.g., EfficientNetB0).")
        base_model.trainable = True
        x = GlobalAveragePooling2D()(base_model.output)
        x = Dropout(0.4)(x)
        x = Dense(256, activation='relu')(x)
        # final dense should have dtype float32 when using mixed precision
        outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)
        model = Model(inputs=base_model.input, outputs=outputs)
        model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
        model.summary(print_fn=lambda s: print(s))
        return model

    def run_dl_analysis(self, batch_size=8, epochs=30, test_size=0.2, random_state=42):
        print("\n" + "="*80 + "\n🚀 STARTING DEEP LEARNING PIPELINE\n" + "="*80)

        print("\n[STEP 1/2] Loading and Preparing Images...")
        images, labels = self.load_images()
        if len(images) == 0:
            print("❌ No images found. Exiting.")
            return

        num_classes = len(self.class_names)
        y_categorical = to_categorical(labels, num_classes=num_classes)

        # Use ImageDataGenerator with rescale to avoid double normalization later
        datagen = ImageDataGenerator(rescale=1.0/255.0)
        # Split: we will normalize using the generator later; for simplicity we normalize arrays now
        X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
            images, y_categorical, np.arange(len(images)),
            test_size=test_size, random_state=random_state, stratify=labels
        )

        # Free memory of the original arrays where possible
        del images, labels

        print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")
        print(f"Classes: {self.class_names}")

        print("\n[STEP 2/2] Training and Evaluating EfficientNetB4...")
        dl_model = self.build_efficientnet_b4_model(num_classes=num_classes)

        # Training data generator (with augmentations)
        train_gen = ImageDataGenerator(
            rotation_range=20,
            horizontal_flip=True,
            vertical_flip=False,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            rescale=1.0/255.0
        ).flow(X_train, y_train, batch_size=batch_size, shuffle=True)

        # Validation data is normalized the same way
        X_test_norm = X_test.astype('float32') / 255.0

        callbacks = [
            EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
            ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, verbose=1)
        ]

        steps_per_epoch = max(1, len(X_train) // batch_size)

        history = dl_model.fit(
            train_gen,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            validation_data=(X_test_norm, y_test),
            callbacks=callbacks,
            verbose=1
        )

        print("\nGenerating Final Reports...")
        self.plot_learning_curve(history, 'EfficientNetB4')

        test_loss, test_acc = dl_model.evaluate(X_test_norm, y_test, verbose=0)
        print(f"\n🏆 Final EfficientNetB4 Test Accuracy: {test_acc*100:.2f}%")

        y_pred_probs = dl_model.predict(X_test_norm)
        y_true = np.argmax(y_test, axis=1)
        y_pred = np.argmax(y_pred_probs, axis=1)

        self.generate_evaluation_report(y_true, y_pred, 'EfficientNetB4')

        model_path = os.path.join(OUTPUT_DIR, 'EfficientNetB4_final.h5')
        dl_model.save(model_path)
        print(f"✅ Model saved to {model_path}")


if __name__ == "__main__":
    # 1. Initialize dataset_path to a safe default.
    dataset_path = "PlantVillage"
    
    try:
        if kagglehub is not None:
            print("Downloading dataset from Kaggle Hub (via kagglehub)...")
            # The download might fail, but dataset_path already has the default value.
            download_root_path = kagglehub.dataset_download("emmarex/plantdisease")
            
            # This line only executes if the download succeeds.
            dataset_path = os.path.join(download_root_path, 'PlantVillage')
        else:
            print(f"ℹ Using local dataset path: {dataset_path}")

        # Use the finalized dataset_path (either downloaded or local default)
        classifier = PlantDiseaseDL(base_path=dataset_path, img_size=(224, 224))
        classifier.run_dl_analysis(batch_size=4, epochs=30) # Using batch_size=4 and 224x224 to resolve previous OOM errors.
        
    except Exception as e:
        print(f"❌ AN UNEXPECTED ERROR OCCURRED: {e}")
        traceback.print_exc()


✅ GPU available: 2 GPU(s) configured.
✅ Mixed precision policy set to 'mixed_float16'.
✅ XLA JIT enabled.

🚀 STARTING DEEP LEARNING PIPELINE

[STEP 1/2] Loading and Preparing Images...
Found 15 classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
Loaded 20638 images.
Train samples: 16510, Test samples: 4128
Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 

I0000 00:00:1761300718.706663     256 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1761300718.707278     256 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast (Cast)         │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ cast[0][0]        │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │

I0000 00:00:1761300786.429468     290 service.cc:148] XLA service 0x791c28007e40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761300786.430000     290 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1761300786.430019     290 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1761300786.457281     290 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1761300786.548403     290 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4127/4127 ━━━━━━━━━━━━━━━━━━━━ 505s 75ms/step - accuracy: 0.6656 - loss: 1.0788 - val_accuracy: 0.7846 - val_loss: 0.7650 - learning_rate: 1.0000e-04
Epoch 2/30
4127/4127 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.7500 - loss: 0.6335 - val_accuracy: 0.7885 - val_loss: 0.7433 - learning_rate: 1.0000e-04
Epoch 3/30
4127/4127 ━━━━━━━━━━━━━━━━━━━━ 209s 51ms/step - accuracy: 0.9157 - loss: 0.2624 - val_accuracy: 0.7607 - val_loss: 1.0577 - learning_rate: 1.0000e-04
Epoch 4/30
4127/4127 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 1.0000 - loss: 0.0901 - val_accuracy: 0.7645 - val_loss: 1.0351 - learning_rate: 1.0000e-04
Epoch 5/30
4127/4127 ━━━━━━━━━━━━━━━━━━━━ 210s 51ms/step - accuracy: 0.9484 - loss: 0.1609 - val_accuracy: 0.4385 - val_loss: 2.8182 - learning_rate: 1.0000e-04
Epoch 6/30
   1/4127 ━━━━━━━━━━━━━━━━━━━━ 3:10 46ms/step - accuracy: 1.0000 - loss: 0.1997
Epoch 6: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
4127/4127 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step -